# Compare piano transcription methods

In [1]:
from src import utils
from src.detect.onset_utils import OnsetMaker, bandpass_filter
from src.detect.midi_utils import group_onsets
from pretty_midi import PrettyMIDI
import librosa
from joblib import Parallel, delayed
import numpy as np
import pandas as pd
import src.visualise.visualise_utils as vutils

In [3]:
corp = utils.CorpusMaker.from_excel('corpus_updated', only_annotated=True, only_30_corpus=False)

## Current approach

In [12]:
loaded = utils.load_corpus_from_files(f'{utils.get_project_root()}/data/cambridge-jazz-trio-database-v02')
fns = set(track['fname'] for track in corp.tracks)
res_cur = [{'track': t.item['fname'], 'method': 'current', **t.item['validation']['piano']} for t in loaded if t.item['validation']['piano'] is not None and t.item['fname'] in fns]

## Automatic MIDI transcription

In [21]:
def auto_midi(item):
    om = OnsetMaker(item, skip_processing=True)
    mm = PrettyMIDI(f'{utils.get_project_root()}/data/cambridge-jazz-trio-database-v02/{item["fname"]}/piano_midi.mid')
    ons = [o.start for o in mm.instruments[0].notes]
    fmt = group_onsets(ons, keep_func=np.min)
    return {
        'track': item['fname'],
        'method': 'automatic_midi', 
        **om.compare_onset_detection_accuracy(
            fname=rf'{om.references_dir}/manual_annotation/{item["fname"]}_piano.txt',
            onsets=fmt,
        )
    }

In [22]:
# Fast enough to process consecutively
res_mm = [auto_midi(i) for i in corp.tracks]

## Spectral flux

In [23]:
# Set the optimised defaults
params_sf = {
    "wait": 18,
    "delta": 0.023021937161684,
    "pre_max": 20,
    "post_max": 22,
    "pre_avg": 53,
    "post_avg": 4
}

In [24]:
def spec_flux(item):
    made = OnsetMaker(item, skip_processing=False)
    ons = librosa.onset.onset_detect(
        y=made.audio['piano'],
        sr=utils.SAMPLE_RATE,
        hop_length=utils.HOP_LENGTH,
        units='time',
        **params_sf
    )  
    return {
        'track': item['fname'],
        'method': 'spectral_flux', 
        **made.compare_onset_detection_accuracy(
            fname=rf'{made.references_dir}/manual_annotation/{item["fname"]}_piano.txt',
            onsets=ons,
        )
    }

In [25]:
with Parallel(n_jobs=-1, backend='loky', verbose=10) as par:
    res_sf = par(delayed(spec_flux)(i) for i in corp.tracks)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 24 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of  34 | elapsed:   14.3s remaining:  2.5min
/home/hwc31/Cambridge-Jazz-Trio-Database/src/detect/onset_utils.py:173: UserWarning: item evansb-nocovernominimum-koticktmotianp-1956-f675dc4a, track bass exceeds silence threshold: (0.8 > 0.33)
  warnings.warn(
[Parallel(n_jobs=-1)]: Done   7 out of  34 | elapsed:   28.7s remaining:  1.8min
[Parallel(n_jobs=-1)]: Done  11 out of  34 | elapsed:   37.9s remaining:  1.3min
/home/hwc31/Cambridge-Jazz-Trio-Database/src/detect/onset_utils.py:173: UserWarning: item powellb-sweetgeorgiabrown-russellcroachm-1950-25a7592c, track bass exceeds silence threshold: (0.9 > 0.33)
  warnings.warn(
[Parallel(n_jobs=-1)]: Done  15 out of  34 | elapsed:   44.9s remaining:   56.8s
[Parallel(n_jobs=-1)]: Done  19 out of  34 | elapsed:   49.6s remaining:   39.1s
[Parallel(n_jobs=-1)]: Done  23 out of  34 | elapsed:   52.0s remaining:   24.9s
[Parallel(n

## Current approach, no filtering

In [26]:
def no_filter(item):
    made = OnsetMaker(item, skip_processing=True)
    fname = made._get_channel_override_fpath('piano', made.instrs['piano'])
    made.audio = {}
    y, _ = librosa.load(
        path=fname,
        sr=utils.SAMPLE_RATE,
        mono=True,
        offset=0,
        duration=None,
        dtype=np.float64,
        res_type='soxr_vhq',
    )
    made.audio['piano'] = librosa.util.normalize(y)
    ons = made.onset_detect_cnn('piano')
    return {
        'track': item['fname'],
        'method': 'cnn_no_filter', 
        **made.compare_onset_detection_accuracy(
            fname=rf'{made.references_dir}/manual_annotation/{item["fname"]}_piano.txt',
            onsets=ons,
        )
    }

In [27]:
with Parallel(n_jobs=-1, backend='loky', verbose=10) as par:
    res_nf = par(delayed(no_filter)(i) for i in corp.tracks)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 24 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of  34 | elapsed:    2.4s remaining:   25.1s
[Parallel(n_jobs=-1)]: Done   7 out of  34 | elapsed:    4.7s remaining:   18.1s
[Parallel(n_jobs=-1)]: Done  11 out of  34 | elapsed:    6.6s remaining:   13.8s
[Parallel(n_jobs=-1)]: Done  15 out of  34 | elapsed:    8.4s remaining:   10.7s
[Parallel(n_jobs=-1)]: Done  19 out of  34 | elapsed:    9.0s remaining:    7.1s
[Parallel(n_jobs=-1)]: Done  23 out of  34 | elapsed:   10.5s remaining:    5.0s
[Parallel(n_jobs=-1)]: Done  27 out of  34 | elapsed:   10.7s remaining:    2.8s
[Parallel(n_jobs=-1)]: Done  31 out of  34 | elapsed:   11.0s remaining:    1.1s
[Parallel(n_jobs=-1)]: Done  34 out of  34 | elapsed:   12.2s finished


## Current approach, more filter

In [28]:
def more_filter(item, lowcut, highcut):
    made = OnsetMaker(item, skip_processing=True)
    fname = made._get_channel_override_fpath('piano', made.instrs['piano'])
    made.audio = {}
    y, _ = librosa.load(
        path=fname,
        sr=utils.SAMPLE_RATE,
        mono=True,
        offset=0,
        duration=None,
        dtype=np.float64,
        res_type='soxr_vhq',
    )
    y = bandpass_filter(
        audio=y,
        lowcut=lowcut,
        highcut=highcut,
        order=made.order
    )
    made.audio['piano'] = librosa.util.normalize(y)
    ons = made.onset_detect_cnn('piano')
    return {
        'track': item['fname'],
        'method': f'cnn_{lowcut}_{highcut}', 
        **made.compare_onset_detection_accuracy(
            fname=rf'{made.references_dir}/manual_annotation/{item["fname"]}_piano.txt',
            onsets=ons,
        )
    }

In [29]:
with Parallel(n_jobs=-1, backend='loky', verbose=10) as par:
    res_220_1760 = par(delayed(more_filter)(i, 220, 1760) for i in corp.tracks)
    res_27_4186 = par(delayed(more_filter)(i, 27.5, 4186) for i in corp.tracks)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 24 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of  34 | elapsed:    2.2s remaining:   22.8s
[Parallel(n_jobs=-1)]: Done   7 out of  34 | elapsed:    4.5s remaining:   17.2s
[Parallel(n_jobs=-1)]: Done  11 out of  34 | elapsed:    6.9s remaining:   14.5s
[Parallel(n_jobs=-1)]: Done  15 out of  34 | elapsed:    8.2s remaining:   10.4s
[Parallel(n_jobs=-1)]: Done  19 out of  34 | elapsed:    9.5s remaining:    7.5s
[Parallel(n_jobs=-1)]: Done  23 out of  34 | elapsed:   10.2s remaining:    4.9s
[Parallel(n_jobs=-1)]: Done  27 out of  34 | elapsed:   10.5s remaining:    2.7s
[Parallel(n_jobs=-1)]: Done  31 out of  34 | elapsed:   11.0s remaining:    1.1s
[Parallel(n_jobs=-1)]: Done  34 out of  34 | elapsed:   11.8s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 24 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of  34 | elapsed:    2.4s remaining:   24.4s
[Parallel(n_jobs=-1)]: Done   7 out of  34 | e

## Putting it all together

In [6]:
def fmt(x):
    x = x.dropna()
    return f'{round(np.mean(x), 2)} ± {round(np.std(x), 2)}'

In [47]:
df = pd.DataFrame(res_mm + res_sf + res_nf + res_220_1760)

In [76]:
f = df.groupby('method')['f_score'].apply(fmt)
p = df.groupby('method')['precision'].apply(fmt)
r = df.groupby('method')['recall'].apply(fmt)
(
    pd.concat([f, p, r], axis=1)
    .transpose()
    .rename(
        columns={'automatic_midi': "(1)", 'spectral_flux': "(2)", 'cnn_no_filter': "(3)", 'cnn_220_1760': "(4)"},
        index={'f_score': 'F', 'precision': 'P', 'recall': 'R'}
    )
    [["(1)", "(2)", "(3)", "(4)"]]
)

method,(1),(2),(3),(4)
F,0.78 ± 0.13,0.83 ± 0.06,0.91 ± 0.04,0.91 ± 0.04
P,0.71 ± 0.16,0.79 ± 0.1,0.9 ± 0.06,0.95 ± 0.03
R,0.86 ± 0.09,0.89 ± 0.05,0.93 ± 0.04,0.88 ± 0.06


## F-score table for copying

In [4]:
loaded = utils.load_corpus_from_files(f'{utils.get_project_root()}/data/cambridge-jazz-trio-database-v02')
fns = set(track['fname'] for track in corp.tracks)
bigres = [pd.DataFrame([{'track': t.item['fname'], 'instr': ins, **t.item['validation'][ins]} for t in loaded if t.item['validation'][ins] is not None and t.item['fname'] in fns]) for ins in ['piano', 'bass', 'drums', 'mix', 'mix_downbeats']]
fdf = pd.concat(bigres)

In [26]:
f = fdf.groupby('instr')['f_score'].apply(fmt)
p = fdf.groupby('instr')['precision'].apply(fmt)
r = fdf.groupby('instr')['recall'].apply(fmt)
(
    pd.concat([f, p, r], axis=1)
    .transpose()
    .rename(
        columns={'bass': "Bass", 'drums': "Drums", 'piano': "Piano", 'mix': "Beats", "mix_downbeats": "Downbeats"},
        index={'f_score': 'F', 'precision': 'P', 'recall': 'R'}
    )
)

instr,Bass,Drums,Beats,Downbeats,Piano
F,0.92 ± 0.08,0.94 ± 0.04,0.95 ± 0.08,0.6 ± 0.45,0.92 ± 0.03
P,0.93 ± 0.04,0.95 ± 0.05,0.97 ± 0.05,0.6 ± 0.45,0.93 ± 0.04
R,0.91 ± 0.11,0.93 ± 0.05,0.94 ± 0.12,0.6 ± 0.45,0.92 ± 0.05


In [71]:
big = pd.DataFrame(utils.CorpusMaker.from_excel('corpus_updated', only_annotated=False, only_30_corpus=False).tracks)
mbz = big['mbz_id'].unique()

In [64]:
every = pd.DataFrame(utils.CorpusMaker.from_excel('corpus_updated', only_annotated=False, only_30_corpus=False, keep_all_tracks=True).tracks)

In [75]:
brushes = every[(every['notes'].str.contains('brushes')) & (~every['mbz_id'].isin(mbz))]
brushes

,track_name,album_name,recording_year,in_30_corpus,bandleader,pianist,channel_overrides,mbz_id,notes,time_signature,first_downbeat,rating_bass_audio,rating_bass_detection,rating_drums_audio,rating_drums_detection,rating_mix,rating_piano_audio,rating_piano_detection,rating_comments,has_annotations,links,excerpt_duration,timestamps,log,musicians,photos,fname
9,Beautiful Love,The Oracle,1989,False,Dave Holland,Hank Jones,{},b1e0ce61-70cc-4497-b7c3-96d3d5398bf2,brushes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,{'external': [nan]},,"{'start': NaT, 'end': NaT}",[],"{'pianist': 'Hank Jones', 'bassist': 'Dave Hol...","{'musicians': {'pianist': None, 'bassist': Non...",hollandd-beautifullove-joneshhigginsb-1989-b1e...
14,Mayas Dance,The Oracle,1989,False,Dave Holland,Hank Jones,{},88e2ebcd-2a44-482b-9964-12ddf6c4bbf1,brushes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,{'external': [nan]},,"{'start': NaT, 'end': NaT}",[],"{'pianist': 'Hank Jones', 'bassist': 'Dave Hol...","{'musicians': {'pianist': None, 'bassist': Non...",hollandd-mayasdance-joneshhigginsb-1989-88e2ebcd
22,Direct Input,Encounters,1988,False,Dave Holland,Mark Isaacs,{},e0f26f4f-f358-40e5-b7c7-5407706c29c0,brushes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,{'external': [nan]},,"{'start': NaT, 'end': NaT}",[],"{'pianist': 'Mark Isaacs', 'bassist': 'Dave Ho...","{'musicians': {'pianist': None, 'bassist': Non...",hollandd-directinput-isaacsmhaynesr-1988-e0f26f4f
40,Wedding March,Travellin Man,1969,False,Stanley Cowell,Stanley Cowell,{},d9681f0c-6ddf-4b98-96e8-e36431834e46,brushes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,{'external': [nan]},,"{'start': NaT, 'end': NaT}",[],"{'pianist': 'Stanley Cowell', 'bassist': 'Stev...","{'musicians': {'pianist': None, 'bassist': Non...",cowells-weddingmarch-novoselshoppsj-1969-d9681f0c
44,I Think Its Time to Say Goodbye Again,Sienna,1989,False,Stanley Cowell,Stanley Cowell,{},dc965b1f-eb2b-4490-a15a-acc4d0490dde,brushes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,{'external': [nan]},,"{'start': NaT, 'end': NaT}",[],"{'pianist': 'Stanley Cowell', 'bassist': 'Ron ...","{'musicians': {'pianist': None, 'bassist': Non...",cowells-ithinkitstimeto-mcclurercopelandk-1989...
60,Bright Passion,Bright Passion,1993,False,Stanley Cowell,Stanley Cowell,{},a39a7d7e-f68d-42b6-be88-4a47b7964e30,brushes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,{'external': [nan]},,"{'start': NaT, 'end': NaT}",[],"{'pianist': 'Stanley Cowell', 'bassist': 'Chey...","{'musicians': {'pianist': None, 'bassist': Non...",cowells-brightpassion-thomascthomasw-1993-a39a...
66,Piano Concerto No 1 Serenity,Bright Passion,1993,False,Stanley Cowell,Stanley Cowell,{},37c18b2e-ef56-4951-985b-b053c4728fb6,brushes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,{'external': [nan]},,"{'start': NaT, 'end': NaT}",[],"{'pianist': 'Stanley Cowell', 'bassist': 'Chey...","{'musicians': {'pianist': None, 'bassist': Non...",cowells-pianoconcertono1serenity-thomascthomas...
69,Bright Passion,Live at Copenhagen Jazz House,1993,False,Stanley Cowell,Stanley Cowell,{},89f7f03a-b0d6-4dac-88bb-e3ffb74fa486,brushes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,{'external': [nan]},,"{'start': NaT, 'end': NaT}",[],"{'pianist': 'Stanley Cowell', 'bassist': 'Chey...","{'musicians': {'pianist': None, 'bassist': Non...",cowells-brightpassion-thomascthomasw-1993-89f7...
79,Blue Lou,The Man Complete Recordings 19461959,1956,False,Ray Brown,Hank Jones,{},675abd81-d2db-4d72-ade7-e2784cef0176,brushes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,{'external': [nan]},,"{'start': NaT, 'end': NaT}",[],"{'pianist': 'Hank Jones', 'bassist': 'Ray Brow...","{'musicians': {'pianist': None, 'bassist': Non...",brownr-bluelou-joneshrichb-1956-675abd81
80,Song of the Volga Boatmen,The Man Complete Recordings 19461959,1956,False,Ray Brown,Hank Jones,{},c7fe2c5c-7bdb-4a01-8977-b6c6e9641243,brushes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,{'external': [nan]},,"{'start': NaT, 'end': NaT}",[],"{'pianist': 'Han